# 图像分割
1、**简介**

- 利用图像的灰度、颜色、纹理、形状等特征，将图像分成若干个互不重叠的区域，并使这些特征在同一区域内呈现相似性，在不同区域之间存在明显的差异性，然后就可以将分割的图像中具有独特性质的区域提取出来用于不同研究。

2、**图像分割应用**

- **机车检验领域**。通过应用到轮毂裂纹图像的分割，几十发现裂纹，保证行车安全；
- **生物医学工程邻域**。对肝脏CT图像进行分割，为临床治疗和病理学研究提供帮助等。

3、**常用图像分割方法**

- **阈值分割**：对**图像灰度值**进行度量，设置不同类别的阈值，达到分割的目的；
- **边缘分割**：对**图像边缘**进行检测，即检测图像中灰度值发生跳变的地方，则为一片区域的边缘；
- **直方图法**：对**图像的颜色**建立直方图，而直方图的波峰波谷能够表示一块区域的颜色值的范围，来达到分割的目的；
- **特定理论**：基于**聚类分析**、**小波变换**等理论完成图像分割。

# 案例：基于聚类的整图（bull.jpg）分割（K-means）
1、**目标**：

- 利用K-means聚类算法对**图像像素点颜色**进行聚类实现简单的图像分割

2、**输出**：

- 同一聚类中的点使用相同颜色标记，不同聚类颜色不同

### 1、建立工程，导入sklearn相关包

In [2]:
import numpy as np
# 用于加载创建图片
import PIL.Image as image
from sklearn.cluster import KMeans

### 2、加载图片并进行预处理
1、**PIL.Image子库补充**：

- **image.open(fp, mode = 'r')**：用于打开/标识给定图像；
 - 参数fp表示文件名称/文件路径；
 - 参数mode表示打开方式
 
- **image.getpixel(xy)**：返回指定位置的像素值；
 - 参数xy以(x, y)的形式给定
 
- **image.putpixel(xy, value)**：调整指定位置的像素值，对于单波段图像则只需给定一个单个数值型数值即可，而对于多波段图像则需要给定一个像素值构成的元组；
 - 参数xy以(x, y)形式给定，用于指定要调整的位置；
 - 参数value，用于指定该位置的像素值
 - **注意：**由于Pillow库版本升级，目前参数value只能传入整型数据，即须使用**int**对数据进行转换后才可以使用
 
2、**NumPy库补充**：

- **np.mat(data, dtype = None)**：用于将输入的数据data转换为矩阵，与**np.asmatrix**相同
 - 参数data：表示要转换的数据
 - 参数dtype：用于指定换换后的数据类型

In [19]:
def loadData(filepath):
    # 以二进制形式打开文件
    f = open(filepath, 'rb')

    # 设定存储图片像素值的初始化空列表
    data = []
    
    # 以列表形式获得图片的像素值
    img = image.open(f)
       
    # 获得图片的像素大小
    m, n = img.size
   
    # 将每个像素点的RGB颜色处理到0-1
    for i in range(m):
        for j in range(n):
            x, y, z = img.getpixel((i, j))
            data.append([x / 256.0, y / 256.0, z / 256.0])
    f.close()
   
    # 以矩阵形式返回存储图片像素点的data列表以及图片大小
    return np.mat(data), m, n

bull_filepath = r'D:\学习资料\Python项目\案例数据\bull.jpg'
bull_imgdata, bull_row, bull_col = loadData(bull_filepath)
bull_imgdata, bull_imgdata.shape, bull_row, bull_col

(matrix([[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]), (230400, 3), 640, 360)

### 3、加载KMeans聚类算法

In [20]:
# 调用KMeans聚类方法，并设置聚类中心为3
km = KMeans(n_clusters = 3)

### 4、对像素点进行聚类并输出
- 依据设定的聚类中心，对属于同一聚类的点使用同样的颜色进行标记

In [21]:
# 聚类获得每个像素所属的类别
label = km.fit_predict(bull_imgdata)
label = label.reshape((bull_row, bull_col))

# 创建一张新的灰度图保存聚类后的结果
pic_new = image.new('L', (bull_row, bull_col))

# 根据所属类别向图片中添加灰度值
for i in range(bull_row):
    for j in range(bull_col):
        
        # 由于版本更新，putpixel方法参数value目前只支持整型int
        pic_new.putpixel((i, j), int(256 / (label[i][j] + 1)))

# 以JPEG格式保存图像
pic_new.save('KMeans-bull.jpg', 'JPEG')

# 基于聚类算法的整图（person.jpg）分割（K-means）
- **注意**：对person.jpg图像进行分割时，采用分割bull.jpg图像的代码将会出现异常：ValueError: too many values to unpack(excepted 3)，意为：有太多值而目前只有三个变量，无法匹配打开。主要是由于：由于person.jpg图像的RGB颜色有4个维度，而bull.jpg图像的RGB颜色只有三个维度，因此使用原来代码时RGB颜色赋值变量只有三个，数值与变量个数不匹配，因而出现该错误。

In [22]:
# 导入相关包
import numpy as np
import PIL.Image as image
from sklearn.cluster import KMeans

In [23]:
# 加载数据
def loadData(filepath):
    # 以 二进制形式打开图像文件
    f = open(filepath, 'rb')

    # 设置存放图像像素值的空列表
    data = []

    # 以列表形式获得图像像素值
    img = image.open(f)

    # 获得图像的大小，即像素179 * 180
    m, n = img.size

    # 将每个像素点的颜色处理为0-1
    for i in range(m):
        for j in range(n):
            a, b, c, d = img.getpixel((i, j))
            data.append([a / 256.0, b / 256.0, c / 256.0, d / 256.0])
    f.close()

    return np.asmatrix(data), m, n

person_filepath = r'D:\学习资料\Python项目\案例数据\person.jpg'
person_imgdata, person_row, person_col = loadData(person_filepath)
person_imgdata, person_imgdata.shape, person_row, person_col

(matrix([[ 0.21875   ,  0.71875   ,  0.890625  ,  0.99609375],
         [ 0.140625  ,  0.6953125 ,  0.8828125 ,  0.99609375],
         [ 0.140625  ,  0.6953125 ,  0.8828125 ,  0.99609375],
         ..., 
         [ 0.140625  ,  0.6953125 ,  0.8828125 ,  0.99609375],
         [ 0.140625  ,  0.6953125 ,  0.875     ,  0.99609375],
         [ 0.140625  ,  0.6953125 ,  0.875     ,  0.99609375]]),
 (32220, 4),
 179,
 180)

In [24]:
# 调用算法
km = KMeans(n_clusters = 4)

# 以一维数组形式聚类获得每个像素点所属类别
label = km.fit_predict(person_imgdata)

# 将一维数组转化为与原图像相同的维度
label = label.reshape((person_row, person_col))

# 创建一个新的灰度图保存聚类结果
pic_new = image.new('L', (person_row, person_col))

# 根据每个像素点所属类别向该图中添加灰度值
for i in range(person_row):
    for j in range(person_col):
        pic_new.putpixel((i, j), int(256 / (label[i][j] + 1)))

# 保存结果
pic_new.save('K-means-person.jpg', 'JPEG')